## Chapter 12: Inheritance for good or for worse

### Subclassing built-in types is tricky

In [2]:
#The problem
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)  # __init__ inherited from dict, ignored setitem
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
dd.update(three=3) ## update inherited from dict, ignored setitem
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [6]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
ad['a']

42

In [8]:
d = {}
d.update(ad)  # dict.update ignore __getitem__
d

{'a': 'foo'}

In [10]:
# Using collections.UserDict solve these issues.
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [11]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [13]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [14]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict2(a='foo')
ad['a']

42

In [15]:
d = {}
d.update(ad)
d['a']

42

In [16]:
d

{'a': 42}

### Multiple inheritance and method resolution order(mro)  

In [19]:
#Diamond problem
class A:
    def ping(self):
        print('ping', self)
        
class B(A):
    def pong(self):
        print('pong', self)

class C(A):
    def pong(self):
        print('PONG', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping', self)
    
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.ping(self)
        
d = D()
d.pong()

pong <__main__.D object at 0x10fc86358>


In [20]:
C.pong(d)

PONG <__main__.D object at 0x10fc86358>


In [21]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [22]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [23]:
print_mro(bool)  #bool inherits methods and attributes from int and object.

bool, int, object


In [24]:
print_mro(dict)

dict, object


In [25]:
print_mro(list)

list, object


In [26]:
import tkinter
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object
